In [ ]:
import pandas as pd
import numpy as np

# 1) Load Excel (first sheet) — use relative path inside notebooks/
raw_path = "/Users/amlim/triathlon-performance/data/Strava Running Data.xlsx"
df = pd.read_excel(raw_path, sheet_name=0)

# 2) Normalize column names
df.columns = (df.columns
              .str.strip()
              .str.lower()
              .str.replace(r"[^a-z0-9]+", "_", regex=True)
              .str.strip("_"))

print("Raw cols:", df.columns.tolist()[:25])

# 3) Parse datetime (local date/time as strings)
# Try parsing with/without timezone info, then drop tz
dt = pd.to_datetime(df.get("start_date_local"), errors="coerce", utc=True)
if dt.notna().any():
    dt = dt.dt.tz_convert(None)
else:
    dt = pd.to_datetime(df.get("start_date_local"), errors="coerce")

df["date_local"] = dt.dt.date.astype(str)
df["start_time_local"] = dt.dt.strftime("%H:%M:%S")

# 4) Core metrics
# Distance (m -> km), duration (s -> min)
df["distance_km"] = df.get("distance", np.nan) / 1000.0
df["duration_min"] = df.get("moving_time", np.nan) / 60.0
df["elev_gain_m"]  = df.get("total_elevation_gain", np.nan)

# Average speed (m/s) if provided; else compute from distance/time
avg_speed_mps = df.get("average_speed")
if avg_speed_mps is None or avg_speed_mps.isna().all():
    # compute if both distance_km and duration_min present
    with np.errstate(divide="ignore", invalid="ignore"):
        avg_speed_mps = (df["distance_km"] * 1000.0) / (df["duration_min"] * 60.0)
df["avg_speed_mps"] = avg_speed_mps

# Pace (sec/km) and (min/km); compute robustly regardless of source
with np.errstate(divide="ignore", invalid="ignore"):
    df["avg_pace_s_per_km"] = np.where(df["avg_speed_mps"] > 0,
                                       1000.0 / df["avg_speed_mps"],
                                       (df["duration_min"] * 60.0) / (df["distance_km"] * 1000.0))
df.loc[~np.isfinite(df["avg_pace_s_per_km"]), "avg_pace_s_per_km"] = np.nan
df["avg_pace_min_per_km"] = df["avg_pace_s_per_km"] / 60.0

# 5) Indoor/outdoor heuristic (GPS presence -> outdoor)
def infer_indoor(row):
    val = row.get("start_latlng")
    has_gps = pd.notna(val) and "[" in str(val)
    return "outdoor" if has_gps else "indoor"
df["indoor_outdoor"] = df.apply(infer_indoor, axis=1)

# 6) Time-of-day buckets
def bucket_tod(tstr):
    try:
        h = int(str(tstr).split(":")[0])
        if 5 <= h <= 11:   return "Morning"
        if 12 <= h <= 17:  return "Afternoon"
        return "Evening"
    except Exception:
        return np.nan
df["time_of_day"] = df["start_time_local"].apply(bucket_tod)

# 7) Effectiveness (lower pace = better) — top 25% fastest = 1
thr = df["avg_pace_min_per_km"].quantile(0.25)
df["high_effectiveness_run"] = (df["avg_pace_min_per_km"] <= thr).astype(int)

# 8) Select cleaned columns (pick only those that actually exist)
candidate_keep = [
    "date_local", "start_time_local", "timezone",
    "duration_min", "distance_km", "elev_gain_m",
    "avg_pace_min_per_km",
    "indoor_outdoor", "time_of_day",
    "high_effectiveness_run",
    "type", "sport_type"
]
keep = [c for c in candidate_keep if c in df.columns]
clean = df[keep].copy()

# 9) Save cleaned CSV (relative path)
clean_path = "/Users/amlim/triathlon-performance/data/cleaned_running.csv"
clean.to_csv(clean_path, index=False)
print(f"Saved: {clean_path}  rows: {len(clean)}  cols: {len(clean.columns)}")
print("Clean cols:", clean.columns.tolist())
clean.head(3)